In [ ]:
! pip install transforms

In [ ]:
! pip install langchain_groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 9.0 MB/s eta 0:00:00


In [ ]:
! pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00


In [ ]:
! pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
!pip install transformers accelerate torch torchvision pillow --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 867.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 27.1 MB/s eta 0:00:00


In [ ]:
%%writefile function.py

from transformers import BlipProcessor, BlipForConditionalGeneration,DetrImageProcessor, DetrForObjectDetection
from PIL import Image
import torch

def get_image_caption(image_path):
    image = Image.open(image_path).convert('RGB')
    model_name = "Salesforce/blip-image-captioning-base"
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    processor = BlipProcessor.from_pretrained(model_name)
    model = BlipForConditionalGeneration.from_pretrained(model_name).to(device)

    inputs = processor(images=image, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_new_tokens=20)
    caption = processor.tokenizer.decode(output[0], skip_special_tokens=True)

    return caption

def detect_imge (image_path):

    image = Image.open(image_path).convert('RGB')
    processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
    model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

    detections = ""
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        detections += '[{} {} {}]'.format(int(box[0]),int(box[1]),int(box[2]),int(box[3]))
        detections += '{}'.format(model.config.id2label[int(label)])
        detections += '{}\n'.format(float(score))
    return detections



image_path = '/content/aa_picture_20150818_6080682_web.jpg'
detections = detect_imge(image_path)
print("detections:", detections)


Writing function.py


In [ ]:
%%writefile tools.py

from langchain.tools import BaseTool
from transformers import (
    BlipProcessor, BlipForConditionalGeneration,
    DetrImageProcessor, DetrForObjectDetection
)
from PIL import Image
import torch

                                                            # ------------------ Image Caption Tool ------------------

class ImageCaptionTool(BaseTool):
    name: str = "Image captioner"
    description: str = (
        "Use this tool when given the path to an image that you would like to be described. "
        "It will return a simple caption describing the image."
    )

    def _run(self, img_path: str) -> str:
        image = Image.open(img_path).convert('RGB')
        model_name = "Salesforce/blip-image-captioning-large"
        device = "cuda" if torch.cuda.is_available() else "cpu"

        processor = BlipProcessor.from_pretrained(model_name)
        model = BlipForConditionalGeneration.from_pretrained(model_name).to(device)

        inputs = processor(image, return_tensors='pt').to(device)
        output = model.generate(**inputs, max_new_tokens=20)

        caption = processor.tokenizer.decode(output[0], skip_special_tokens=True)
        return caption

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")


                                                            # ------------------ Object Detection Tool ------------------

class ObjectDetectionTool(BaseTool):
    name: str = "Object detector"
    description: str = (
        "Use this tool when given the path to an image that you would like to detect objects. "
        "It will return a list of all detected objects. Each element in the list in the format: "
        "[x1, y1, x2, y2] class_name confidence_score."
    )

    def _run(self, img_path: str) -> str:
        image = Image.open(img_path).convert('RGB')

        processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
        model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

        inputs = processor(images=image, return_tensors="pt")
        outputs = model(**inputs)

        target_sizes = torch.tensor([image.size[::-1]], dtype=torch.float)
        results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

        detections = ""
        for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
            detections += '[{}, {}, {}, {}]'.format(
                int(box[0]), int(box[1]), int(box[2]), int(box[3])
            )
            detections += ' {}'.format(model.config.id2label[int(label)])
            detections += ' {:.2f}\n'.format(float(score))

        return detections

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")


Writing tools.py


In [ ]:
%%writefile app.py

from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from tempfile import NamedTemporaryFile
import streamlit as st
import os
from langchain_groq import ChatGroq
from tools import ImageCaptionTool, ObjectDetectionTool
from google.colab import userdata


st.title('Ask a question about your Image')
st.header('Upload your Image')
file = st.file_uploader('Upload an image', type=['png', 'jpg', 'jpeg'], label_visibility="collapsed")


tools = [ImageCaptionTool(),ObjectDetectionTool()]

convertional_memory = ConversationBufferMemory(
    memory_key="chat_history", k=5, return_messages=True)




LLM = ChatGroq(api_key='api', model='llama3-70b-8192', temperature=0)


agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=LLM,
    verbose=True,
    max_iterations=3,
    memory=convertional_memory,
    early_stopping_method='generate'
)


if file:
    st.image(file, use_container_width=True)
    user_question = st.text_input('Ask a question about your image')
    with NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
         temp_file.write(file.getvalue())
         temp_file.flush()
         image_path = temp_file.name

         if user_question and user_question.strip() != '':
            with st.spinner(text='in progress...'):
                response = agent.run(f"The image is located at {image_path}. {user_question}")
                st.write("🤖 Answer:")
                st.write(response)


Overwriting app.py


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("2tABXL52IcMbMGedO4FLKAJ3E5E_2aiz2ghYGY46C2RrNKEhZ")

public_url = ngrok.connect(8501)
print("App is live at:", public_url)


App is live at: NgrokTunnel: "https://5ae411a29894.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.85.141.245:8501

2025-08-06 00:50:01.568099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754441401.597203    3315 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754441401.605551    3315 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-06 00:50:01.632965: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebui

  Stopping...
  Stopping...
^C
